In [1]:
import numpy as np
import pandas as pd

from utils.taxonomic_hierarchical_distance import score
from utils.utils import map_label_to_idx

df = pd.read_csv("./data/train/annotations.csv")

df, label_map = map_label_to_idx(df, "label")

print(label_map)

annotations_in_label_map = list(label_map.keys())

solution_dict = {}
submission_dict = {}

index = 0
for soln_annotation in annotations_in_label_map:
    for submission_annotation in annotations_in_label_map:
        solution_dict[index] = soln_annotation
        submission_dict[index] = submission_annotation
        index += 1

# Turn the dictionaries into dataframes with columns 
# annotation_id for keys and concept_name for values
solution_df = pd.DataFrame(list(solution_dict.items()), columns=["annotation_id", "concept_name"])
submission_df = pd.DataFrame(list(submission_dict.items()), columns=["annotation_id", "concept_name"])

# Drop index column for both dataframes
solution_df = solution_df.reset_index(drop=True)
submission_df = submission_df.reset_index(drop=True)

final_score, scores = score(
    solution=solution_df,
    submission=submission_df,
    row_id_column_name="annotation_id",
    concept_name="concept_name",
    return_scores=True,
)

np_scores = np.array(scores)
np_scores = np_scores.reshape(len(annotations_in_label_map), len(annotations_in_label_map))

# Normalize the scores across the rows
norm_np_scores = np_scores / np_scores.sum(axis=1, keepdims=True)

# Add one to the diagonal of the score matrix
np_scores_ones = np_scores + np.eye(np_scores.shape[0])
norm_np_score_ones = np_scores_ones / np_scores_ones.sum(axis=1, keepdims=True)

print(np.sum(norm_np_scores, axis=1))
print(np.sum(norm_np_score_ones, axis=1))

# Save np_scores to npy files
np.save("./cfg/hierarchy/score_table", np_scores)
np.save("./cfg/hierarchy/norm_score_table", norm_np_scores)

# Save np_scores_ones to npy files
np.save("./cfg/hierarchy/score_table_ones", np_scores_ones)
np.save("./cfg/hierarchy/norm_score_table_ones", norm_np_score_ones)

{'Sebastolobus': 0, 'Apostichopus leukothele': 1, 'Scotoplanes': 2, 'Keratoisis': 3, 'Munnopsidae': 4, 'Scotoplanes globosa': 5, 'Chionoecetes tanneri': 6, 'Hippasteria': 7, 'Munidopsis': 8, 'Serpulidae': 9, 'Delectopecten': 10, 'Crinoidea': 11, 'Tunicata': 12, 'Asteroidea': 13, 'Pandalus amplus': 14, 'Isidella tentaculum': 15, 'Holothuroidea': 16, 'Paragorgia arborea': 17, 'Heterochone calyx': 18, 'Paragorgia': 19, 'Terebellidae': 20, 'Ophiuroidea': 21, 'Peniagone': 22, 'Actiniaria': 23, 'Ceriantharia': 24, 'Isididae': 25, 'Rathbunaster californicus': 26, 'Paelopatides confundens': 27, 'Abyssocucumis abyssorum': 28, 'Strongylocentrotus fragilis': 29, 'Porifera': 30, 'Psolus squamatus': 31, 'Liponema brevicorne': 32, 'Farrea': 33, 'Caridea': 34, 'Hexactinellida': 35, 'Actinernus': 36, 'Pannychia moseleyi': 37, 'Gastropoda': 38, 'Benthocodon pedunculata': 39, 'Octopus rubescens': 40, 'Microstomus pacificus': 41, 'Heteropolypus ritteri': 42, 'Parastenella': 43, 'Staurocalyptus': 44, 'Hor

In [2]:
import numpy as np
import torch
from utils.cost_weighted_ce import ConfidenceLossWithLogits

n_batches = 16
n_heads = 4
classes = 10

criterion = ConfidenceLossWithLogits()

# Generate random predictions
output_preds = torch.randn(n_heads, n_batches, classes)
# Generate random targets
target = torch.randint(0, 2, (n_batches, classes)).float()
# Generate random confidence scores
confidence_scores = torch.rand(n_batches)

losses = torch.stack([criterion(output_pred, target, confidence_scores) for output_pred in output_preds])

print(losses)

inputs: torch.Size([16, 10])
targets: torch.Size([16, 10])
confidence: torch.Size([16])
ce_loss: torch.Size([16])
inputs: torch.Size([16, 10])
targets: torch.Size([16, 10])
confidence: torch.Size([16])
ce_loss: torch.Size([16])
inputs: torch.Size([16, 10])
targets: torch.Size([16, 10])
confidence: torch.Size([16])
ce_loss: torch.Size([16])
inputs: torch.Size([16, 10])
targets: torch.Size([16, 10])
confidence: torch.Size([16])
ce_loss: torch.Size([16])
tensor([5.6575, 6.0832, 5.9018, 5.8532])
